### 分类：概率生成模型（Probabilistic Generative Model）

分类是指：输入$x$, 经过一个分类模块后，输出$x$的类别。

![](./ml_png/function1.png)

典型的案例有：

![](./ml_png/case.png)

其中本节课要介绍的例子：首先宝可梦有很多类，要做的是

输入$x$(给定一只宝可梦)，经过一个分类模块，输出$x$(宝可梦)所属的类别。

![](./ml_png/catogary.png)

其中这里的输入$x$需要是一个具体的量化指标，指标如下：

    HP(生命值)、Attack(进攻)、Defense(防御)、SP Atk(特殊进攻)、SP Def(特殊防御)、Speed(速度)


理想的方案选择（Ideal alternative）

** Function(Model)

![](./ml_png/indealFunction.png)
   
** loss function:
   $$ L(f) = \sum_{n} \delta \left( f \left( x^n  \right) \ne \hat{y}^n \right) $$
   
    该函数表示了在训练集中分类错误的samples data的次数。
   
** find the best function

    如：Perceptron, SVM(能解决，但本章节不使用这两种方法)
    
**如何进行分类？**

1. 首先准备训练数据：$(x^1, \hat{y}^1),(x^2, \hat{y}^2),\dots,(x^N, \hat{y}^N)$

2. 分类函数的选择

**思考**：能否用regression的方式来进行分类？首先用二分类的例子来进行说明。

在训练时，定义第一类的结果为1，第二类的结果为-1. 测试时，regression结果接近1，则代表它应该归类为第一类; regression结果接近-1，归类为第二类。

隐含的一个问题是，如果两类数据是理想化情况，即分布均匀且噪声较低，这是可行的。但如果中间有噪声数据，导致某一类的预测值远大于1或远小于1，此时的loss function为了减小损失值，分类平面会发生偏移，从而更好地迎合更小的loss。但发生偏移后，可能会导致分类准确度下降。如图。

![](./ml_png/class_to_regress.png)

进行三分类时，将第一类分为标签1，第二类分为标签2，第三类分为标签3.也会是同样的情况。此时各类间是否会被误认为有相关性的高低。如第一、二类之间的间距是1，第一第三类的间距是2，他们是否代表了一、二之间的相关性比一、三之间的相关性高。如果将第一类分为标签1，第二类分为标签2，第三类分为标签5，按照这种分类情况呢？这种间距应该怎么选择才更合适。

关于如何进行类别预测，**先考虑一个概率问题**。

![](./ml_png/boxes_probability.png)

如图，有两个盒子，Box1和Box2,两个盒子分别装有蓝球和青球若干个。从其中一个盒子取球时，一般会分两步，第一步，从两个盒子中选一个。其中取第一个盒子的概率是$p\left( B_1 \right)=\frac{2}{3}$, 取第二个盒子的概率是$p\left( B_2 \right)=\frac{1}{3}$。第二步，在选择的盒子中选球。若选中第一个盒子，那么从BOX1中选中蓝球的概率是$p\left( Blue | B_1 \right)=\frac{4}{5}$, 从BOX1中选中青球的概率是$p\left( Green | B_1 \right)=\frac{1}{5}$；同理，在BOX2中选中蓝球的概率是$p\left( Blue | B_2 \right)=\frac{2}{5}$，在BOX2中选中青球的概率是$p\left( Green | B_2 \right)=\frac{3}{5}$.

现在给定了一个蓝球（对象），要估测这个球来自Box1和Box2的概率，根据贝叶斯公式

$$
p(B_1 | Blue) = \frac{p(Blue | B_1) p(B_1) }{p(Blue | B_1) p(B_1) + p(Blue | B_2) p(B_2)} = \frac 45
$$

$$
p(B_2 | Blue) = \frac{p(Blue | B_2) p(B_2) }{p(Blue | B_1) p(B_1) + p(Blue | B_2) p(B_2)}  = \frac 15 
$$



现在假设Box1是class1，box2是class2，class1、class2内部不再是颜色完全一致的小球，而是特征相近的物体的归类。如图

![](./ml_png/classes_probability.png)

之前估测某个小球是从某盒子取出来的概率，那么现在变为估计某物体x从哪一类$C_1、C_2$取出来的概率（概率越大，那么代表从该类取出来的可能性越大）。或者更准确地说，哪一类的特征与该物体的特征更接近，那么该类就包含了该物体。于是，估计$x$属于类别$C_1$的概率用数学公式可表达为：

$$
p(C_1 | x) = \frac{p(x | C_1) p(C_1) }{p(x | C_1) p(C_1) + p(x | C_2) p(C_2)}
$$

其中分母$p(x) = p(x | C_1) p(C_1) + p(x | C_2) p(C_2)$称为生成模型（Generative model）.

具体回到宝可梦的例子中来，$p(C_1)$和$p(C_2)$可直接在training data中计算得来，如$C_1$是Water水系的79个宝可梦，$C_2$是Normal普通系的61个宝可梦，训练数据中提供的两种系的宝可梦有多少个，则分别计算他们的比率，就是这里的$p(C_1)$和$p(C_2)$，它们被称为先验prior。

![](./ml_png/prior.png)

然后计算$p(x|C_1)$,即从$C_1$中挑选出$x$的可能性，使用宝可梦的例子来说明就是从水系挑选出杰尼龟的可能性。

![](./ml_png/pxc1.png)

**具体应该怎么计算？**

首先给定一系列training数据，这些数据均用列向量来表示，列向量中的元素就是每个宝可梦的attribution的具体量化. 只考虑Defense和SP Denfense两个attribution，则water水系的宝可梦用点图来表示如图

![](./ml_png/water_type.png)

假设这些数据根据高斯分布采样（其他采样方式也行）所得，其中高斯分布公式为

$$
f_{\mu,\Sigma}\left(𝑥 \right)=\frac{1}{(2 \pi)^{D/2}} \frac{1}{|\Sigma|^{1/2}} \exp \Bigl\{−\frac 12 \left(𝑥−\mu \right)^T \Sigma^{−1} \left(𝑥−\mu \right)\Bigr\}
$$

根据这个公式，可知只有均值$\mu$,协方差$\Sigma$两个参数。但是任意的$\mu, \Sigma$所构成的gaussian distribution都有可能采样出这些samples,应该如何挑选出一对适合的参数值？挑选的一个原则是，参数对应的高斯分布能最大化地采样出大部分数据，如下图。一共包含了79个training data, 左侧的高斯分布相对于右侧的高斯分布能samples得到更多的数据点。

![](./ml_png/twoMeanConvariance.png)

而这个能最大化采样数据的方法是Maximum Likelihood(极大似然法)，具体做法是

$$
L\left( \mu,\Sigma \right)=f_{\mu,\Sigma} \left(x^1 \right) f_{\mu,\Sigma} \left(x^2 \right) f_{\mu,\Sigma} \left(x^3 \right)\dots f_{\mu,\Sigma} \left(x^{79} \right)
$$

对该函数最大化，

$$ \mu^*,\Sigma^* = \arg \max_{\mu, \Sigma}L\left( \mu, \Sigma \right) $$

求解得，$\mu^* = \frac{1}{79}{\sum_{n=1}^{79}{x^n}}, \Sigma^*=\frac{1}{79}{\sum_{n=1}^{79}{\left(x^n-\mu^*\right)\left( x^n-\mu^* \right)^T}}$

将training data中的数据分别代入，求解得Water水系以及Normal普通系的宝可梦的均值和协方差为

![](./ml_png/water_normal_mean_corv.png)

将求得的值分别代回高斯分布公式中，有

$$
f_{\mu^1,\Sigma^1} \left( x \right)=
\frac{1}{(2\pi)^{D/2}} \frac{1}{{|\Sigma^1 |}^{1/2}}  \exp \Bigl\{−\frac 12 \left(x−\mu^1 \right)^T \left(\Sigma^1 \right)^{−1} \left(x−\mu^1 \right) \Bigr\} 
$$


$$
f_{\mu^2,\Sigma^2} \left( x \right)=
\frac{1}{(2\pi)^{D/2}} \frac{1}{{|\Sigma^2 |}^{1/2}}  \exp \Bigl\{−\frac 12 \left(x−\mu^2 \right)^T \left(\Sigma^2 \right)^{−1} \left(x−\mu^2 \right) \Bigr\} 
$$

进一步地，将给定的新的测试集$x=\left[\begin{matrix} x_1 \\  x_2 \end{matrix} \right]$ 代入到 $f_{\mu^1,\Sigma^1} \left( x \right)$中，可得到$p\left(x|C_1 \right)$；同理代入$f_{\mu^2,\Sigma^2} \left( x \right)$，得$p\left(x|C_2 \right)$，于是$p\left(C_2 | x \right)$中的各项均已求得。根据求得的值，可知输入的$x$属于哪一类。

![](./ml_png/probabilityOFwho.png)


**分类结果如何？**

首先将testing data可视化，如图

![](./ml_png/dataVisualization.png)

根据以上方法，只用defense和SP defense两个attribution来计算概率$p(C_1|x)$，分类精度仅有47%(相当于盲猜水平)

![](./ml_png/twoAttriResult.png)

与在线性回归linear regression中相同的策略，刚开始时只用两个特征，当结果不满意时，进一步使用更多的特征（6个），此时$\mu^1, \mu^2$都是6维的Vector, $\Sigma^1, \Sigma^2$都是$6 \times 6$的matrix。分类结果如下,精度提高到64%.

![](./ml_png/sixAttriResult.png)

**进一步提高testing results的方法**

共享同一个协方差矩阵，令$\Sigma^1 = \Sigma^2 = \Sigma$，减少参数，有

![](./ml_png/modifying_model.png)

分类结果如图，

![](./ml_png/sameCorv.png)

需要注意的是，分类平面是一个直线线段，分类精度为54%。同时考虑将$\Sigma$值置相同值，六个attribution同时考虑，分类精度提高到73%.

### 贝叶斯公式的转化

上面所介绍的内容都在围绕着贝叶斯公式$p(C_1 | x)$中每一项的内容的计算。将该式子作进一步的转换，有

$$\begin{align}
p(C_1 | x) &= \frac{p(x | C_1) p(C_1) }{p(x | C_1) p(C_1) + p(x | C_2) p(C_2)} \\
&= \frac{1}{1+\exp{ \left(-z \right)}}  \longrightarrow  z = \ln \frac{p(x | C_1) p(C_1)}{p(x | C_2) p(C_2)} \\
&= \sigma \left( z \right),
\end{align}$$

其中$\sigma(z)$为sigmoid function. 

![](./ml_png/sigmoidFunction.png)

进一步地，将$z$中的各项值代入具体算式，有

$$\begin{align}
z &= \ln \frac{p(x | C_1) p(C_1)}{p(x | C_2) p(C_2)}  \\
&= \ln \frac{p(x | C_1)}{p(x | C_2) } + \ln \frac{ p(C_1)}{p(C_2)} \\
&= \ln \frac{\frac{1}{(2\pi)^{D/2}} \frac{1}{{|\Sigma^1 |}^{1/2}}  \exp \Bigl\{−\frac 12 \left(x−\mu^1 \right)^T \left(\Sigma^1 \right)^{−1} \left(x−\mu^1 \right) \Bigr\} } { \frac{1}{(2\pi)^{D/2}} \frac{1}{{|\Sigma^2 |}^{1/2}}  \exp \Bigl\{−\frac 12 \left(x−\mu^2 \right)^T \left(\Sigma^2 \right)^{−1} \left(x−\mu^2 \right) \Bigr\}  } + \ln \frac{\frac{ N_1}{(N_1 + N_2)}} {\frac{N_2}{(N_1 + N_2)}} \\
&= \ln \frac{{|\Sigma^2 |}^{1/2}}{{|\Sigma^1 |}^{1/2}} + \frac 12 \Bigl\{  \left(x−\mu^2 \right)^T \left(\Sigma^2 \right)^{−1} \left(x−\mu^2 \right) - \left(x−\mu^1 \right)^T \left(\Sigma^1 \right)^{−1} \left(x−\mu^1 \right) \Bigr\} + \ln \frac{N_1}{N_2}  \\
&= \ln \frac{{|\Sigma^2 |}^{1/2}}{{|\Sigma^1 |}^{1/2}} +
\frac 12 x^T (\Sigma^2)^{-1} x -  (\mu^2)^T \left(\Sigma^2 \right)^{−1} x + \frac 12 \left(\mu^2 \right)^T (\Sigma^2)^{-1} (\mu)^2  -
\frac 12 x^T (\Sigma^1)^{-1} x +  (\mu^1)^T \left(\Sigma^1 \right)^{−1} x - \frac 12 \left(\mu^1 \right)^T (\Sigma^1)^{-1} (\mu)^1  +
\ln \frac{N_1}{N_2}  \\
\end{align}$$

令$\Sigma_1 = \Sigma_2 = \Sigma$， 有
$$\begin{align}
z &= 
(\mu^2)^T \left(\Sigma \right)^{−1} x + \frac 12 \left(\mu^2 \right)^T (\Sigma)^{-1} (\mu)^2  - 
(\mu^1)^T \left(\Sigma \right)^{−1} x - \frac 12 \left(\mu^1 \right)^T (\Sigma)^{-1} (\mu)^1  +
\ln \frac{N_1}{N_2}  \\
&= (\mu^2 - \mu^1)^T \Sigma^{-1} x + \frac 12 (\mu^2)^T (\Sigma)^{-1} \mu^2 - \frac 12 (\mu^1)^T (\Sigma)^{-1} \mu^1 + \ln \frac{N_1}{N_2} \\
&= w^Tx + b, \ \mathrm{where} \ w^T = (\mu^2 - \mu^1)^T \Sigma^{-1}, \ \ 
b = \frac 12 (\mu^2)^T (\Sigma)^{-1} \mu^2 - \frac 12 (\mu^1)^T (\Sigma)^{-1} \mu^1 + \ln \frac{N_1}{N_2}
\end{align}$$

亦即有$z = w \cdot x + b $，这就解释了为甚么上面的分解平面变成了线性平面。

进一步地，$p(C_1 | x) = \sigma(z) = \sigma( w \cdot x + b)$,为了计算出这个$w$和$b$,花费了相当大的力气，经过了以下步骤：

1. 根据sample得到的数据进行分布的预测（如将data的分布预估为高斯分布）

2. 将分布的parameters一一找出来，如$\mu_1, \mu_2, \Sigma$, 然后计算出$w, b$

3. 将$w, b$回代到$p(C_1 | x) = \sigma(z)$中，完成一个目标的类别估计。

由于大部分的时间都花费在了参数的计算上，那么为了缩短计算时间，有没有办法可以直接预估出$w,b$的值，然后估算目标所属类别呢？这就是logistics regression要解决的问题。